In [12]:
from infoseq.bpe import bpe_from_text

from infoseq.dataset import kjv_path

with open(kjv_path, 'rb') as f:
    kjv = f.read()

len(kjv)

4457889

In [13]:
bpe = bpe_from_text(kjv, max_tokens=5000)
len(bpe)

with open('/tmp/bpe.json', 'w') as f:
    f.write(bpe.to_json())

In [9]:
[k for k in bpe.encode_map if len(k) > 1][:10]

[]

In [14]:
with open('/tmp/bpe.dot', 'w') as f:
    f.write(bpe.to_dot())